# Train and test AVIDNet

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
# Installs PyTorch, PyTorch/XLA, and Torchvision
# Copy this cell into your own notebooks to use PyTorch on Cloud TPUs 
# Warning: this may take a couple minutes to run
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

In [3]:
from google.colab import drive
drive.mount('/content/drive')
ROOT_DIR = '/content/drive/My Drive/01 Research/COVID-19/AVIDNet_latest/AVIDNet'
%cd $ROOT_DIR

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/01 Research/COVID-19/AVIDNet_latest/AVIDNet


In [4]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp

import os
from collections import defaultdict

from datasets.utils import create_datasets, get_data_loaders, show_samples
from models.model import save_model, load_model
from models.train import train_models
from models.utils import set_random_seed

## Create datasets

In [5]:
from google.colab import auth
auth.authenticate_user()

In [6]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1974  100  1974    0     0  73111      0 --:--:-- --:--:-- --:--:-- 70500
OK
17 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.32.0).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.


https://ostrokach.gitlab.io/post/google-colab-storage/

In [7]:
!mkdir data_from_bucket
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 avidnet data_from_bucket

mkdir: cannot create directory ‘data_from_bucket’: File exists
Using mount point: /content/drive/My Drive/01 Research/COVID-19/AVIDNet_latest/AVIDNet/data_from_bucket
Opening GCS connection...
Mounting file system...
File system has been successfully mounted.


In [8]:
cfg = {
    'purpose' : 'fixmatch',
    'images_dir' : './data_from_bucket/data/CXR/ori',
    'data_dir' : './data/CXR',
    'out_dir': './data/trained_models',
    'num_labeled': 22,
    'mu' : 1,
    'lambda_u': 1.0,
    'threshold': 0.95,
    'fold' : 1,
    'epochs' : 20,
    'batch_size': 8,
    'random_seed': 0,
    'scheduler': 'step',
    'overwrite': False,
    'print_to_file': False,
    'use_tpu': True,
    'is_finetuning': False,
    'metric_types' : ['acc', 'ppv', 'recall', 'f1'],
    'dataset_types': ['train', 'test']
}
create_datasets(cfg, cfg['images_dir'])

Dataset is already exists.


Test samples

In [9]:
# sample_loaders, _, class_names = get_data_loaders(dataset_type='test', cfg=cfg,
#                                                   fold_id=0, overwrite=True)
# print(class_names)
# show_samples(sample_loaders['test'], class_names, iter_num=3, mu=cfg['mu'])

## Train the model

In [10]:
# !cat models/train.py

In [11]:
random_seed = 0
set_random_seed(random_seed)

xmp.spawn(train_models, args=(cfg,), nprocs=8, start_method='fork')

Process 0 is using TPU:0
--------------------
Model 1
--------------------

Epoch 0/19
----------
[test]
LOSS - 1.0906
ACC - 0.3200
PPV - ALL: 0.1081  COVID-19: 0.0000  PNEUMONIA: 0.0000  NORMAL: 0.3243 
RECALL - ALL: 0.3333  COVID-19: 0.0000  PNEUMONIA: 0.0000  NORMAL: 1.0000 
F1 - ALL: 0.1633  COVID-19: 0.0000  PNEUMONIA: 0.0000  NORMAL: 0.4898 

Epoch 1/19
----------
[test]
LOSS - 1.0997
ACC - 0.3300
PPV - ALL: 0.4425  COVID-19: 0.0000  PNEUMONIA: 1.0000  NORMAL: 0.3276 
RECALL - ALL: 0.3433  COVID-19: 0.0000  PNEUMONIA: 0.0300  NORMAL: 1.0000 
F1 - ALL: 0.1839  COVID-19: 0.0000  PNEUMONIA: 0.0583  NORMAL: 0.4935 

Epoch 2/19
----------
[test]
LOSS - 1.0369
ACC - 0.3933
PPV - ALL: 0.7190  COVID-19: 1.0000  PNEUMONIA: 0.8000  NORMAL: 0.3569 
RECALL - ALL: 0.4067  COVID-19: 0.0200  PNEUMONIA: 0.2000  NORMAL: 1.0000 
F1 - ALL: 0.2951  COVID-19: 0.0392  PNEUMONIA: 0.3200  NORMAL: 0.5261 

Epoch 3/19
----------


Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 872, in _try_get_data
    data = self._data_queue.get(timeout=timeout)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 105, in get
    raise Empty
queue.Empty

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch_xla/distributed/parallel_loader.py", line 141, in _loader_worker
    _, data = next(data_iter)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 435, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1068, in _next_d

KeyboardInterrupt: ignored

## Save and load the model

In [ ]:
# out_dir = f'trained_models/{cfg["purpose"]}'
# save_model(trained_models, out_dir, cfg)

In [ ]:
# trained_models = load_model(cfg)

## Reference

- https://tutorials.pytorch.kr/beginner/saving_loading_models.html
- https://stackoverflow.com/questions/42480111/model-summary-in-pytorch

## Evaluate the model - WIP (Work In Process)

In [ ]:
# def evaluate_model(model, criterion, optimizer):
#     since = time.time()

#     model.eval()   # Set model to evaluate mode

#     CLS = {0: 'COVID-19', 1: 'Pneumonia', 2: 'Normal'}
#     running_loss = 0.0
#     running_corrects = 0
#     acc, size, fp, fn = defaultdict(int), defaultdict(int), defaultdict(int), defaultdict(int)

#     # Iterate over data.
#     for inputs, labels in dataloaders['test']:
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward
#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         loss = criterion(outputs, labels)

#         # statistics
#         running_loss += loss.item() * inputs.size(0)
#         running_corrects += torch.sum(preds == labels.data)
#         for pred, label in zip(preds, labels.data):
#             size[CLS[label.item()]] += 1
#             if pred == label.data:
#                 acc[CLS[label.item()]] += 1
#             else:
#                 fp[CLS[pred.item()]] += 1
#                 fn[CLS[label.item()]] += 1

#         iter_loss = running_loss / dataset_sizes['test']
#         iter_acc = running_corrects.double() / dataset_sizes['test']

#     print('Loss: {:.4f} Acc: {:.4f}'.format(iter_loss, iter_acc))
#     accs = {c: round(float(n) / size[c], 4) for c, n in acc.items()}
#     ppv = {c: round(float(n) / (acc[c] + sum([s for c_temp, s in fp.items()
#                                               if c_temp != c])), 4)
#            for c, n in acc.items()}
#     recall = {c: round(float(n) / (acc[c] + sum([s for c_temp, s in fn.items()
#                                                  if c_temp != c])), 4)
#               for c, n in acc.items()}
#     print(f'[ACC] COVID-19: {accs["COVID-19"]}, Pnuemonia: {accs["Pneumonia"]}, Normal: {accs["Normal"]}')
#     print(f'[PPV] COVID-19: {ppv["COVID-19"]}, Pnuemonia: {ppv["Pneumonia"]}, Normal: {ppv["Normal"]}')
#     print(f'[Recall] COVID-19: {recall["COVID-19"]}, Pnuemonia: {recall["Pneumonia"]}, Normal: {recall["Normal"]}')

#     time_elapsed = time.time() - since
#     print('Test complete in {:.0f}m {:.0f}s'.format(
#         time_elapsed // 60, time_elapsed % 60))

In [ ]:
# evaluate_model(model_ft, criterion, optimizer_ft)